<a href="https://colab.research.google.com/github/yang1325/AI_selfpraticenote/blob/main/tensorflow_covid/tensorflow_Coviddata_splitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#載入所需模組
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import load
from numpy import save
import random
import os
import gc
from datetime import datetime, timezone, timedelta
tz = timezone(timedelta(hours=+8)) 

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(299,299, 1)))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
#cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
#cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Flatten())
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(4, activation="softmax"))
cnn_model.summary()
gc.collect()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 297, 297, 128)     1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 512)       1180160   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 512)      0

393

In [ ]:
path = "/content/gdrive/My Drive/splited_data"
checkpoint_path = "/content/gdrive/My Drive/trained/model(1).ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#cnn_model.compile(optimizer="Adam",
#              loss=tf.keras.losses.categorical_crossentropy,
#              metrics=['accuracy'])

cnn_model.load_weights(checkpoint_path)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()
def loss(model, x, y, training):
  y_ = model(x, training=training)
  return loss_object(y_true=y, y_pred=y_)
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
for k in range(100):
  ran = np.array(random.sample(range(0,9),9))
  for j in range(10):
    if(j<9):
      index = ran[j]
    else:
      index = 9
    print()
    print("now is %d,%d(%d)"%(k,j,index))
    data = load(path + "/%d"%(index) + ".npy")
    n = np.shape(data)[0]
    label = load(path + "/%dy"%(index) + ".npy")
    z = np.apply_along_axis(np.argmax,1,label)
    t = tf.convert_to_tensor(data)
    del data
    gc.collect()
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    if(j==9):
      #cnn_model.evaluate(t[i:(i+1)], z[i:(i+1)])
      for i in range(n):
        if(tem%100 == 0):
          fmt = '%H:%M:%S'
          zoned_time1 = datetime.today().astimezone(tz)
          print(tem,",",zoned_time1.strftime(fmt))
        loss_value, grads = grad(cnn_model, t[i:(i+1)], z[i:(i+1)])
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(z[i:(i+1)], cnn_model(t[i:(i+1)], training=False))
    else:
      #cnn_model.fit(data, label, epochs=1,batch_size = 1,callbacks=[cp_callback])
      index = np.array(random.sample(range(0,n),n))
      tem = 0
      for i in index:
        tem +=  1
        if(tem%100 == 0):
          fmt = '%H:%M:%S'
          zoned_time1 = datetime.today().astimezone(tz)
          print(tem,",",zoned_time1.strftime(fmt))
        loss_value, grads = grad(cnn_model, t[i:(i+1)], z[i:(i+1)])
        optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(z[i:(i+1)], cnn_model(t[i:(i+1)], training=True))
      cnn_model.save_weights(checkpoint_path)
    print("Epoch {},{}: Loss: {:.3f}, Accuracy: {:.3%}".format(k,j,
                 epoch_loss_avg.result(),
                 epoch_accuracy.result()))

    del t
    del label
    gc.collect()


now is 0,0(2)
100 , 22:47:59
200 , 22:48:02
300 , 22:48:05
400 , 22:48:08
500 , 22:48:11
600 , 22:48:14
700 , 22:48:17
800 , 22:48:20
900 , 22:48:23
1000 , 22:48:26
1100 , 22:48:29
1200 , 22:48:32
1300 , 22:48:35
1400 , 22:48:39
1500 , 22:48:42
1600 , 22:48:45
1700 , 22:48:48
1800 , 22:48:51
1900 , 22:48:54
2000 , 22:48:57
2100 , 22:49:00
Epoch 0,0: Loss: 0.378, Accuracy: 86.484%

now is 0,1(6)
100 , 22:49:16
200 , 22:49:19
300 , 22:49:22
400 , 22:49:25
500 , 22:49:29
600 , 22:49:32
700 , 22:49:35
800 , 22:49:38
900 , 22:49:41
1000 , 22:49:44
1100 , 22:49:47
1200 , 22:49:50
1300 , 22:49:53
1400 , 22:49:56
1500 , 22:49:59
1600 , 22:50:03
1700 , 22:50:06
1800 , 22:50:09
1900 , 22:50:12
2000 , 22:50:15
2100 , 22:50:18
Epoch 0,1: Loss: 0.387, Accuracy: 86.531%

now is 0,2(0)
100 , 22:50:37
200 , 22:50:40
300 , 22:50:43
400 , 22:50:46
500 , 22:50:50
600 , 22:50:53
700 , 22:50:56
800 , 22:50:59
900 , 22:51:02
1000 , 22:51:05
1100 , 22:51:08
1200 , 22:51:11
1300 , 22:51:14
1400 , 22:51:17
15